In [2]:
a = '''
Write the code in python for:

done() Download the xml from https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100

From the xml, please parse through to the first download link whose file_type is DLTINS and download the zip
Extract the xml from the zip.
Convert the contents of the xml into a CSV with the following header:
FinInstrmGnlAttrbts.Id
FinInstrmGnlAttrbts.FullNm
FinInstrmGnlAttrbts.ClssfctnTp
FinInstrmGnlAttrbts.CmmdtyDerivInd
FinInstrmGnlAttrbts.NtnlCcy
Issr
Store the csv from step 4) in an AWS S3 bucket
The above function should be run as an AWS Lambda (Optional)'''

In [19]:
import requests
import xml.etree.ElementTree as et

res = requests.get("https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100")
con = res.content
print(res.url)

https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100


In [20]:
with open('data.xml', 'wb') as f:
    f.write(con)
mytree = et.parse('data.xml')

In [128]:
root = mytree.getroot()
print(root.tag,"\n\n", root.attrib,"\n\n", root.text,"\n\n",root.iter)

response 

 {} 

 

 

 <built-in method iter of xml.etree.ElementTree.Element object at 0x000001769D532F20>


In [132]:
root = mytree.getroot()
print(root[0].tag,"\n\n", root[0].attrib,"\n\n", root[0].text,"\n\n",root[0].iter)

lst 

 {'name': 'responseHeader'} 

 
   

 <built-in method iter of xml.etree.ElementTree.Element object at 0x000001769D532E80>


In [133]:
root = mytree.getroot()
print(root[1].tag,"\n\n", root[1].attrib,"\n\n", root[1].text,"\n\n",root[1].iter)

result 

 {'name': 'response', 'numFound': '4', 'start': '0'} 

 
   

 <built-in method iter of xml.etree.ElementTree.Element object at 0x000001769DCC0C20>


In [142]:
root = mytree.getroot()
print(root[0][2][0].tag,"\n\n", root[0][0].attrib,"\n\n", root[0][0].text,"\n\n",root[0][0].iter)

str 

 {'name': 'status'} 

 0 

 <built-in method iter of xml.etree.ElementTree.Element object at 0x000001769DC4B8D0>


In [ ]:
root = mytree.getroot()
print(root.tag,"\n\n", root.attrib,"\n\n", root.text,"\n\n",root.iter)

In [27]:
download_links = root.findall(".//str[@name='download_link']")

In [37]:
for i in download_links:
    i = str(i)
    print(i)

<Element 'str' at 0x000001769DC69210>
<Element 'str' at 0x000001769DD02BB0>
<Element 'str' at 0x000001769DD02ED0>
<Element 'str' at 0x000001769DD03420>


In [40]:
for i in download_links:
    
    if 'DLTINS' in i.text:
        link =i.text
        break

In [41]:
print(link)

http://firds.esma.europa.eu/firds/DLTINS_20210117_01of01.zip


In [46]:
response = requests.get(link)

In [53]:
if response.status_code ==200:
    with open("DLTINS.zip","wb") as f:
        f.write(response.content)
    print("File Dowloaded")
else:
    print("error aa gya bhai")

File Dowloaded


The wb mode stands for "write binary" mode in Python. It is used for writing data to a binary file. When downloading a binary file, such as a zip file, it is important to open the file in binary mode, as this ensures that the data is read and written correctly, without any modification of bytes.

If you open a binary file in text mode (w), Python may perform certain text encoding and decoding operations which can result in corrupted data. Therefore, it is best practice to always open binary files in binary mode.

In [54]:
import zipfile

In [81]:
with zipfile.ZipFile("DLTINS.zip", 'r') as f:
    p =f.namelist()
    print(len(p))
    print(type(p))
    print(*p)
    for i in p:
        name =i
        f.extract(i)

1
<class 'list'>
DLTINS_20210117_01of01.xml


In [83]:
tree = et.parse(name)

In [150]:
print(root.tag,"\n\n", root.attrib,"\n\n", root.text,"\n\n",root.iter)


response 

 {} 

 

 

 <built-in method iter of xml.etree.ElementTree.Element object at 0x000001769D532F20>
